In [ ]:
!pip install -q kaggle
import json

data = {"username":"tunkedsaro","key":"ff3d00dc680ff94caf6f8d4d9bcd3168"}
json_data = json.dumps(data)

# Write the JSON string to a file
file_path = "/content/kaggle.json"
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c signal-processing-sleep-staging-classification
!unzip -q /content/signal-processing-sleep-staging-classification.zip

<hr>

In [5]:
from tqdm import tqdm
import pandas as pd
import os

In [6]:
# Concat every .csv file
filepath = "/content/train/train"
concatdf = []
for i in tqdm(os.listdir(filepath)[:]):
   concatdf.append(pd.read_csv(filepath+'/'+i))
frame = pd.concat(concatdf)

100%|██████████| 83/83 [01:03<00:00,  1.31it/s]


In [7]:
frame

,BVP,ACC_X,ACC_Y,ACC_Z,TEMP,EDA,HR,IBI,Sleep_Stage
0,14.20,23.0,42.0,41.0,34.25,0.231893,78.48,0.8125,N
1,14.10,23.0,42.0,41.0,34.25,0.231893,78.48,0.8125,N
2,13.26,23.0,42.0,41.0,34.25,0.231893,78.48,0.8125,N
3,11.84,23.0,42.0,41.0,34.25,0.231893,78.48,0.8125,N
4,10.07,24.0,41.0,42.0,34.25,0.231893,78.48,0.8125,N
...,...,...,...,...,...,...,...,...,...
1401595,-98.33,15.0,60.0,17.0,34.34,0.093793,73.38,0.8125,W
1401596,-92.57,15.0,59.0,17.0,34.34,0.093793,73.38,0.8125,W
1401597,-81.09,15.0,59.0,17.0,34.34,0.093793,73.38,0.8125,W
1401598,-64.76,17.0,59.0,17.0,34.34,0.093793,73.38,0.8125,W


In [8]:
nr = 1920
new = []
frame = frame
for i in tqdm(range(len(frame)//nr)):
    x = frame[i*nr:(i+1)*nr]
    a = x.groupby(['Sleep_Stage'])[['BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI']].mean()
    a = a.rename(columns={'BVP':'BVPmean','ACC_X':'ACC_Xmean','ACC_Y':'ACC_Ymean','ACC_Z':'ACC_Zmean','TEMP':'TEMPmean','EDA':'EDAmean','HR':'HRmean','IBI':'IBImean'})
    b = x.groupby(['Sleep_Stage'])[['BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI']].std()
    b = b.rename(columns={'BVP':'BVPstd','ACC_X':'ACC_Xstd','ACC_Y':'ACC_Ystd','ACC_Z':'ACC_Zstd','TEMP':'TEMPstd','EDA':'EDAstd','HR':'HRstd','IBI':'IBIstd'})
    ab = a.join(b)
    new.append(ab)
    # break
train_df = pd.concat(new)
train_df

100%|██████████| 66607/66607 [02:58<00:00, 373.08it/s]


,BVPmean,ACC_Xmean,ACC_Ymean,ACC_Zmean,TEMPmean,EDAmean,HRmean,IBImean,BVPstd,ACC_Xstd,ACC_Ystd,ACC_Zstd,TEMPstd,EDAstd,HRstd,IBIstd
Sleep_Stage,,,,,,,,,,,,,,,,
N,-0.161542,25.772917,43.354167,37.964583,34.242667,0.232566,77.705667,0.819922,19.340644,1.023179,0.586084,0.618286,0.013152,0.001059,0.672046,0.053930
W,0.464188,30.088542,42.083333,35.315625,34.290000,0.234968,73.707333,0.880086,38.505130,4.750083,1.627453,3.372847,0.018624,0.002731,1.156732,0.023632
N,-0.334625,36.579167,41.641667,29.467708,34.336333,0.239473,69.774667,0.871899,24.676363,0.536314,0.479636,0.503245,0.019238,0.002010,1.030565,0.030465
N,-0.106750,36.909375,41.536458,29.206250,34.394667,0.245153,68.322667,0.880485,43.919690,0.934050,0.784639,0.671897,0.020456,0.001983,0.131638,0.025990
N,0.171943,36.553125,41.916667,29.185417,34.436667,0.249157,69.521000,0.851082,18.593510,0.578664,0.318499,0.486392,0.016604,0.002465,0.367849,0.027155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W,0.497490,-18.707292,48.331250,23.359375,34.449333,0.120388,65.269667,0.750000,205.665935,27.226163,6.696788,14.692152,0.022207,0.024299,0.623463,0.006053
W,-0.954818,17.207292,53.808333,27.931250,34.438000,0.105932,66.383000,0.799325,77.248217,7.693297,1.970796,3.674548,0.016004,0.007589,1.530591,0.023793
W,0.632328,18.656250,57.406250,20.050000,34.432667,0.103348,71.491000,0.821908,55.448313,1.184652,1.150283,3.047240,0.013403,0.002274,1.586947,0.017525


In [9]:
# Copy index value to "ss" feature
train_df['ss'] = train_df.index
train_df

,BVPmean,ACC_Xmean,ACC_Ymean,ACC_Zmean,TEMPmean,EDAmean,HRmean,IBImean,BVPstd,ACC_Xstd,ACC_Ystd,ACC_Zstd,TEMPstd,EDAstd,HRstd,IBIstd,ss
Sleep_Stage,,,,,,,,,,,,,,,,,
N,-0.161542,25.772917,43.354167,37.964583,34.242667,0.232566,77.705667,0.819922,19.340644,1.023179,0.586084,0.618286,0.013152,0.001059,0.672046,0.053930,N
W,0.464188,30.088542,42.083333,35.315625,34.290000,0.234968,73.707333,0.880086,38.505130,4.750083,1.627453,3.372847,0.018624,0.002731,1.156732,0.023632,W
N,-0.334625,36.579167,41.641667,29.467708,34.336333,0.239473,69.774667,0.871899,24.676363,0.536314,0.479636,0.503245,0.019238,0.002010,1.030565,0.030465,N
N,-0.106750,36.909375,41.536458,29.206250,34.394667,0.245153,68.322667,0.880485,43.919690,0.934050,0.784639,0.671897,0.020456,0.001983,0.131638,0.025990,N
N,0.171943,36.553125,41.916667,29.185417,34.436667,0.249157,69.521000,0.851082,18.593510,0.578664,0.318499,0.486392,0.016604,0.002465,0.367849,0.027155,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W,0.497490,-18.707292,48.331250,23.359375,34.449333,0.120388,65.269667,0.750000,205.665935,27.226163,6.696788,14.692152,0.022207,0.024299,0.623463,0.006053,W
W,-0.954818,17.207292,53.808333,27.931250,34.438000,0.105932,66.383000,0.799325,77.248217,7.693297,1.970796,3.674548,0.016004,0.007589,1.530591,0.023793,W
W,0.632328,18.656250,57.406250,20.050000,34.432667,0.103348,71.491000,0.821908,55.448313,1.184652,1.150283,3.047240,0.013403,0.002274,1.586947,0.017525,W


In [10]:
# Remove axis
train_df.reset_index(drop=True, inplace=True)
train_df

,BVPmean,ACC_Xmean,ACC_Ymean,ACC_Zmean,TEMPmean,EDAmean,HRmean,IBImean,BVPstd,ACC_Xstd,ACC_Ystd,ACC_Zstd,TEMPstd,EDAstd,HRstd,IBIstd,ss
0,-0.161542,25.772917,43.354167,37.964583,34.242667,0.232566,77.705667,0.819922,19.340644,1.023179,0.586084,0.618286,0.013152,0.001059,0.672046,0.053930,N
1,0.464188,30.088542,42.083333,35.315625,34.290000,0.234968,73.707333,0.880086,38.505130,4.750083,1.627453,3.372847,0.018624,0.002731,1.156732,0.023632,W
2,-0.334625,36.579167,41.641667,29.467708,34.336333,0.239473,69.774667,0.871899,24.676363,0.536314,0.479636,0.503245,0.019238,0.002010,1.030565,0.030465,N
3,-0.106750,36.909375,41.536458,29.206250,34.394667,0.245153,68.322667,0.880485,43.919690,0.934050,0.784639,0.671897,0.020456,0.001983,0.131638,0.025990,N
4,0.171943,36.553125,41.916667,29.185417,34.436667,0.249157,69.521000,0.851082,18.593510,0.578664,0.318499,0.486392,0.016604,0.002465,0.367849,0.027155,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66602,0.497490,-18.707292,48.331250,23.359375,34.449333,0.120388,65.269667,0.750000,205.665935,27.226163,6.696788,14.692152,0.022207,0.024299,0.623463,0.006053,W
66603,-0.954818,17.207292,53.808333,27.931250,34.438000,0.105932,66.383000,0.799325,77.248217,7.693297,1.970796,3.674548,0.016004,0.007589,1.530591,0.023793,W
66604,0.632328,18.656250,57.406250,20.050000,34.432667,0.103348,71.491000,0.821908,55.448313,1.184652,1.150283,3.047240,0.013403,0.002274,1.586947,0.017525,W
66605,0.047995,19.400000,57.375000,19.387500,34.402000,0.098031,73.443000,0.811694,56.719350,1.453966,1.185724,2.859184,0.021515,0.002149,0.298866,0.018718,W


<hr>

In [11]:
import lightgbm as lgb

In [12]:
X = train_df.drop(columns=['ss'])
y = train_df['ss']

In [13]:
clf = lgb.LGBMClassifier()
clf.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002417 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4080
[LightGBM] [Info] Number of data points in the train set: 66607, number of used features: 16
[LightGBM] [Info] Start training from score -0.423592
[LightGBM] [Info] Start training from score -2.233797
[LightGBM] [Info] Start training from score -1.434694


LGBMClassifier()

<hr>

In [14]:
submit_df = pd.read_csv("/content/sample_submission.csv")
submit_df

,id,labels
0,test001_00000,W
1,test001_00001,W
2,test001_00002,W
3,test001_00003,NaN
4,test001_00004,NaN
...,...,...
7965,test010_00857,NaN
7966,test010_00858,NaN
7967,test010_00859,NaN
7968,test010_00860,NaN


In [15]:
submit_df = pd.read_csv("/content/sample_submission.csv")
# /content/test/test_segment/test002/test002_00000.csv
# /content/test/test_segment/test002/test002_00001.csv
folder_path = "/content/test/test_segment"
for idx,row in tqdm(submit_df.iterrows()):
    folder = row['id'].split('_')[0]
    file   = row['id'].split('_')[1]
    # print(folder_path+'/'+folder)
    # print(folder_path+'/'+folder+'/'+folder+'_'+file+'.csv')
    csvfile = folder_path+'/'+folder+'/'+folder+'_'+file+'.csv'
    x = pd.read_csv(csvfile)
    x['Sleep_Stage'] = 'xxx'
    a = x.groupby(['Sleep_Stage'])[['BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI']].mean()
    a = a.rename(columns={'BVP':'BVPmean','ACC_X':'ACC_Xmean','ACC_Y':'ACC_Ymean','ACC_Z':'ACC_Zmean','TEMP':'TEMPmean','EDA':'EDAmean','HR':'HRmean','IBI':'IBImean'})
    b = x.groupby(['Sleep_Stage'])[['BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI']].std()
    b = b.rename(columns={'BVP':'BVPstd','ACC_X':'ACC_Xstd','ACC_Y':'ACC_Ystd','ACC_Z':'ACC_Zstd','TEMP':'TEMPstd','EDA':'EDAstd','HR':'HRstd','IBI':'IBIstd'})
    ab = a.join(b)
    # y = x.groupby(['Sleep_Stage'])[['BVP', 'ACC_X', 'ACC_Y', 'ACC_Z', 'TEMP', 'EDA', 'HR', 'IBI']].mean()
    ab.reset_index(drop=True,inplace=True)
    result = clf.predict(ab)
    submit_df.at[idx,'labels'] = result[0]
    # print(result)
    # if idx==5:
    #     break

7970it [00:54, 145.55it/s]


In [16]:
submit_df

,id,labels
0,test001_00000,N
1,test001_00001,N
2,test001_00002,N
3,test001_00003,N
4,test001_00004,N
...,...,...
7965,test010_00857,N
7966,test010_00858,N
7967,test010_00859,N
7968,test010_00860,N


In [17]:
import time
version = '0.0.5'
formatted_time = time.strftime("%H.%M", time.localtime())
formatted_time = str(int(formatted_time.split(".")[0])+7)+formatted_time.split(".")[1]
formatted_time
csv_name = f'v.{version}.{formatted_time}.csv'
print(csv_name)
submit_df.to_csv(csv_name,index=False)

v.0.0.5.2047.csv
